In [ ]:
import openeo
from netCDF4 import Dataset

Remove existing authentication token (if necessary) and then authenticate. Then check which library versions are installed.

In [ ]:
from openeo.rest.auth.config import RefreshTokenStore
RefreshTokenStore().remove()
connection = openeo.connect("openeo.cloud").authenticate_oidc()
connection.list_udf_runtimes()["Python"]["versions"]["3"]["libraries"]

Load datacube:

In [ ]:
s2_cube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent={"west": 4.00, "south": 51.04, "east": 4.10, "north": 51.1},
    temporal_extent=["2022-03-01", "2022-03-31"],
    bands=["B02", "B03", "B04"]
)

Load user-defined function stored in script.R and check installed dependencies.

In [ ]:
udf = openeo.UDF.from_file("script.py")

from openeo.udf.run_code import extract_udf_dependencies
extract_udf_dependencies(udf)

Apply that function to the datacube and download the result.

In [ ]:
rescaled = s2_cube.apply(process=udf)
rescaled.download("apply-udf-scaling.nc")

In [ ]:
# Open the NetCDF file
file_path = "apply-udf-scaling.nc"  # Replace with your NetCDF file path
nc_data = Dataset(file_path, mode='r')

# Print the structure of the NetCDF file
print("NetCDF Dimensions:")
print(nc_data.dimensions)
print("\nNetCDF Variables:")
print(nc_data.variables)

# Iterate through variables and print their details
for var_name, var in nc_data.variables.items():
    print(f"\nVariable: {var_name}")
    print(f"Dimensions: {var.dimensions}")
    print(f"Shape: {var.shape}")
    print(f"Attributes: {var.ncattrs()}")
    print(f"Data (preview): {var[:]}")  # Warning: Large datasets may be overwhelming

# Close the file
nc_data.close()


Run function locally:

In [ ]:
from pathlib import Path
from openeo.udf import execute_local_udf

s2_cube.download('test_input.nc', format='NetCDF')
local_udf = Path('script.py').read_text()
execute_local_udf(local_udf, 'test_input.nc', fmt='netcdf')

<UdfData datacube_list:[<XarrayDataCube shape:(6, 3, 679, 711)>] feature_collection_list:None structured_data_list:None>